In [1]:
import pandas as pd
import numpy as np

# Cargar utilities TX ya procesado
df_tx = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\processed\utilities_tx.csv")

# Cargar subestaciones raw y filtrar TX
df_sub = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\US Electric Power Transmission Substations.csv")
df_sub_tx = df_sub[df_sub["STATE"].astype(str).str.upper().str.strip() == "TX"].copy()

# Limpieza mínima útil para EDA
df_tx["zip"] = df_tx["zip"].astype(str).str.extract(r"(\d+)", expand=False).str.zfill(5)
df_tx["utility_name"] = df_tx["utility_name"].astype(str).str.strip()

rate_cols = ["comm_rate", "ind_rate", "res_rate"]
for c in rate_cols:
    df_tx[c] = pd.to_numeric(df_tx[c], errors="coerce")
df_tx[rate_cols] = df_tx[rate_cols].replace(0, np.nan)

df_sub_tx["LATITUDE"] = pd.to_numeric(df_sub_tx["LATITUDE"], errors="coerce")
df_sub_tx["LONGITUDE"] = pd.to_numeric(df_sub_tx["LONGITUDE"], errors="coerce")
df_sub_tx = df_sub_tx.dropna(subset=["LATITUDE", "LONGITUDE"])

print("✅ df_tx:", df_tx.shape)
print("✅ df_sub_tx:", df_sub_tx.shape)


✅ df_tx: (3085, 10)
✅ df_sub_tx: (5314, 26)


C:\Users\palmi\AppData\Local\Temp\ipykernel_11368\1463773740.py:8: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sub = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\US Electric Power Transmission Substations.csv")


In [2]:
[c for c in df_tx.columns if "county" in c.lower()]


[]

CARGAR DATA

In [3]:
import pandas as pd
import numpy as np

df_tx = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\processed\utilities_tx.csv")
df_sub = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\US Electric Power Transmission Substations.csv")

# Filtrar TX en subestaciones
df_sub_tx = df_sub[df_sub["STATE"].astype(str).str.upper().str.strip() == "TX"].copy()

# Limpieza mínima
df_tx["zip"] = df_tx["zip"].astype(str).str.extract(r"(\d+)", expand=False).str.zfill(5)
df_tx["utility_name"] = df_tx["utility_name"].astype(str).str.strip()

df_sub_tx["ZIP"] = df_sub_tx["ZIP"].astype(str).str.extract(r"(\d+)", expand=False).str.zfill(5)

print("Utilities TX:", df_tx.shape)
print("Substations TX:", df_sub_tx.shape)


Utilities TX: (3085, 10)
Substations TX: (5314, 26)


C:\Users\palmi\AppData\Local\Temp\ipykernel_11368\627781178.py:5: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sub = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\US Electric Power Transmission Substations.csv")


UTILITIES POR ZIP

In [4]:
utilities_by_zip = (
    df_tx.groupby("zip")["utility_name"]
    .nunique()
    .reset_index(name="n_utilities")
)
utilities_by_zip.head()


,zip,n_utilities
0,73301,1
1,73344,1
2,75001,1
3,75002,2
4,75006,2


SUBESTACIONES POR ZIP

In [5]:
substations_by_zip = (
    df_sub_tx.groupby("ZIP")["ID"]
    .count()
    .reset_index(name="n_substations")
    .rename(columns={"ZIP": "zip"})
)
substations_by_zip.head()


,zip,n_substations
0,73949,1
1,75001,2
2,75006,6
3,75007,2
4,75009,5


UNIR TODO POR ZIP

In [6]:
df_zip = utilities_by_zip.merge(substations_by_zip, on="zip", how="left")
df_zip["n_substations"] = df_zip["n_substations"].fillna(0).astype(int)

print(df_zip.shape)
df_zip.head()


(1953, 3)


,zip,n_utilities,n_substations
0,73301,1,0
1,73344,1,0
2,75001,1,2
3,75002,2,0
4,75006,2,6


CREAR SCORE Y RANKING

In [7]:
def minmax(s):
    s = s.astype(float)
    return (s - s.min()) / (s.max() - s.min() + 1e-9)

df_zip["utilities_norm"] = minmax(df_zip["n_utilities"])
df_zip["substations_norm"] = minmax(df_zip["n_substations"])

# pesos simples (podés ajustar después)
df_zip["tmps_score"] = 0.4 * df_zip["utilities_norm"] + 0.6 * df_zip["substations_norm"]

df_zip = df_zip.sort_values("tmps_score", ascending=False)
df_zip["rank"] = range(1, len(df_zip) + 1)

df_zip.head(20)


,zip,n_utilities,n_substations,utilities_norm,substations_norm,tmps_score,rank
1664,79336,4,36,0.500000,0.900,0.740000,1
972,77705,1,40,0.000000,1.000,0.600000,2
1149,78155,5,19,0.666667,0.475,0.551667,3
987,77807,7,10,1.000000,0.250,0.550000,4
1002,77845,6,13,0.833333,0.325,0.528333,5
1657,79323,3,26,0.333333,0.650,0.523333,6
1681,79360,3,25,0.333333,0.625,0.508333,7
1755,79549,2,29,0.166667,0.725,0.501667,8
1778,79735,2,28,0.166667,0.700,0.486667,9
991,77833,6,9,0.833333,0.225,0.468333,10


GUARDAR RANKING FINAL

In [8]:
import os

out_dir = r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\outputs"
os.makedirs(out_dir, exist_ok=True)

df_zip.to_csv(os.path.join(out_dir, "texas_priority_zip_ranking.csv"), index=False)
print("✅ Guardado:", os.path.join(out_dir, "texas_priority_zip_ranking.csv"))


✅ Guardado: C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\outputs\texas_priority_zip_ranking.csv


TOP UTILITIES DENTRO DE LOS ZIP MAS PRIORITARIOS

In [9]:
top_zips = df_zip.head(50)["zip"].tolist()

prospects = (
    df_tx[df_tx["zip"].isin(top_zips)]
    .groupby(["utility_name", "utility_type"])["zip"]
    .nunique()
    .sort_values(ascending=False)
    .reset_index(name="n_top_zips_covered")
)

prospects.head(30)


,utility_name,utility_type,n_top_zips_covered
0,Southwestern Public Service Co,IOU,20
1,South Plains Electric Coop Inc,NON_IOU,12
2,Entergy Texas Inc.,IOU,10
3,"Lyntegar Electric Coop, Inc",NON_IOU,10
4,Guadalupe Valley Elec Coop Inc,NON_IOU,5
5,"Big Country Electric Coop, Inc",NON_IOU,5
6,City of Lubbock - (TX),NON_IOU,5
7,"Bluebonnet Electric Coop, Inc",NON_IOU,5
8,Southwestern Electric Power Co,IOU,5
9,Mid-South Electric Coop Assn,NON_IOU,5


In [10]:
df_zip.shape


(1953, 7)

In [14]:
import os

top_zips = df_zip.head(50)["zip"].astype(str).tolist()

prospects = (
df_tx[df_tx["zip"].astype(str).isin(top_zips)]
.groupby(["utility_name", "utility_type", "ownership"])["zip"]
.nunique()
.sort_values(ascending=False)
.reset_index(name="n_top_zips_covered")
 )

out_dir = r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\outputs"
os.makedirs(out_dir, exist_ok=True)

prospects.to_csv(os.path.join(out_dir, "top_prospects_in_top50_zips.csv"), index=False)
prospects.head(20)


,utility_name,utility_type,ownership,n_top_zips_covered
0,Southwestern Public Service Co,IOU,Investor Owned,20
1,South Plains Electric Coop Inc,NON_IOU,Cooperative,12
2,Entergy Texas Inc.,IOU,Investor Owned,10
3,"Lyntegar Electric Coop, Inc",NON_IOU,Cooperative,10
4,Guadalupe Valley Elec Coop Inc,NON_IOU,Cooperative,5
5,"Big Country Electric Coop, Inc",NON_IOU,Cooperative,5
6,City of Lubbock - (TX),NON_IOU,Municipal,5
7,"Bluebonnet Electric Coop, Inc",NON_IOU,Cooperative,5
8,Southwestern Electric Power Co,IOU,Investor Owned,5
9,Mid-South Electric Coop Assn,NON_IOU,Cooperative,5
